# Fazer

- [X] Classe para feature validation (fit - pega estatisticas dependendo do type, transform - compara estatísticas)
- [ ] implementar redução de memória
- [X] date features
- [ ] implementar decorators para print tempo shape_inicial -> shape_final
- [ ] implementar window features

In [1]:
import sys
import pandas as pd
from sklearn.pipeline import Pipeline

%load_ext autoreload
%autoreload 2
sys.path.insert(1, '../src/')



from guara.feature_engineering.pipeline_modules import *
from guara.feature_engineering.window_features import *
#from feature_engineering_utils import *

import utils

pd.set_option('display.max_columns', None)

In [2]:
df = utils.import_s3_example_dataset('marketing')


df['date'] = pd.date_range(start='1/1/2018', periods=len(df))
df = df.astype({'Education':'category','Marital_Status':'category'}).drop('ID', axis = 1)
df.head(2)

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,idade,dias_cliente,education_n,parceiro,pessoas_casa,renda_per_capta,compra_total_ano,porcentagem_compras_gold,gasto_renda,total_purchases,disc_purchases,num_compras_ano,percent_disc_compra,ticket_medio,conversao_site,cupons_aceitados,date
0,Graduation,Single,10.970592,0,0,58,6.455199,4.488636,6.304449,5.153292,4.488636,4.488636,1.386294,8,2.397895,4,7,0,1,63,2675,2,0,1,10.970592,764.5,0.055959,0.013064,22,1.386294,3.001869,0.136364,5.543906,0.012736,0,2018-01-01
1,Graduation,Single,10.743869,1,1,38,2.484907,0.693147,1.945910,1.098612,0.693147,1.945910,1.098612,1,0.693147,2,5,0,0,66,2125,2,0,3,9.645300,10.5,0.251314,0.000227,4,1.098612,0.687059,0.500000,2.790093,0.002820,0,2018-01-02


In [3]:
train_split = 0.8
n_train_split = int(train_split*len(df))

Xtrain = df.iloc[:n_train_split, :].copy()
Xval = df.iloc[n_train_split:, :].copy()

print(Xtrain.shape)
print(Xval.shape)
Xtrain.head()

(1772, 36)
(444, 36)


,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,idade,dias_cliente,education_n,parceiro,pessoas_casa,renda_per_capta,compra_total_ano,porcentagem_compras_gold,gasto_renda,total_purchases,disc_purchases,num_compras_ano,percent_disc_compra,ticket_medio,conversao_site,cupons_aceitados,date
0,Graduation,Single,10.970592,0,0,58,6.455199,4.488636,6.304449,5.153292,4.488636,4.488636,1.386294,8,2.397895,4,7,0,1,63,2675,2,0,1,10.970592,764.5,0.055959,0.013064,22,1.386294,3.001869,0.136364,5.543906,0.012736,0,2018-01-01
1,Graduation,Single,10.743869,1,1,38,2.484907,0.693147,1.945910,1.098612,0.693147,1.945910,1.098612,1,0.693147,2,5,0,0,66,2125,2,0,3,9.645300,10.5,0.251314,0.000227,4,1.098612,0.687059,0.500000,2.790093,0.002820,0,2018-01-02
2,Graduation,Together,11.179046,0,0,26,6.056784,3.912023,4.852030,4.718499,3.091042,3.761200,0.693147,8,1.098612,10,4,0,0,55,2324,2,1,2,10.485913,367.0,0.055643,0.005112,20,0.693147,3.141136,0.050000,4.769300,0.025490,0,2018-01-03
3,Graduation,Together,10.190432,1,0,26,2.484907,1.609438,3.044522,2.397895,1.386294,1.791759,1.098612,2,0.000000,4,6,0,0,36,2151,2,1,3,9.091895,24.0,0.099091,0.000900,6,1.098612,1.018131,0.333333,3.201632,0.004638,0,2018-01-04
4,PhD,Married,10.973254,1,0,94,5.159055,3.784190,4.779123,3.850148,3.332205,2.772589,1.791759,5,1.386294,6,5,0,0,39,2173,4,1,3,9.874676,203.5,0.036192,0.003485,14,1.791759,2.351588,0.357143,4.472065,0.013711,0,2018-01-05


In [7]:
pipe = Pipeline([ 
    
    #('teste', ValidateDF() ),
    ('union_cat_num', PandasFeatureUnion([ 
        Pipeline([
            ('categoricals',PandasTypeSelector(include = ['category']) ),
            ('label_encoder_Education', LabelEncoderPandas('Education')),
            ('label_encoder_Marital_Status', LabelEncoderPandas('Marital_Status')),
        ]),
        
        Pipeline([
            ('numericals',PandasTypeSelector(exclude = ['category']) ),
        ])
    ])),
    ('drop', ColumnDropper('Kidhome'))

])


----------- PandasTypeSelector


----------- LabelEncoderPandas


----------- LabelEncoderPandas


----------- PandasTypeSelector


----------- ColumnDropper



In [5]:
make_pipeline_graph(pipe)


NameError: name 'make_pipeline_graph' is not defined

In [8]:
pipe.fit_transform(Xtrain)

,Education,Marital_Status,Income,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,idade,dias_cliente,education_n,parceiro,pessoas_casa,renda_per_capta,compra_total_ano,porcentagem_compras_gold,gasto_renda,total_purchases,disc_purchases,num_compras_ano,percent_disc_compra,ticket_medio,conversao_site,cupons_aceitados,date
0,2,2,10.970592,0,58,6.455199,4.488636,6.304449,5.153292,4.488636,4.488636,1.386294,8,2.397895,4,7,0,1,63,2675,2,0,1,10.970592,764.5,0.055959,0.013064,22,1.386294,3.001869,0.136364,5.543906,0.012736,0,2018-01-01
1,2,2,10.743869,1,38,2.484907,0.693147,1.945910,1.098612,0.693147,1.945910,1.098612,1,0.693147,2,5,0,0,66,2125,2,0,3,9.645300,10.5,0.251314,0.000227,4,1.098612,0.687059,0.500000,2.790093,0.002820,0,2018-01-02
2,2,3,11.179046,0,26,6.056784,3.912023,4.852030,4.718499,3.091042,3.761200,0.693147,8,1.098612,10,4,0,0,55,2324,2,1,2,10.485913,367.0,0.055643,0.005112,20,0.693147,3.141136,0.050000,4.769300,0.025490,0,2018-01-03
3,2,3,10.190432,0,26,2.484907,1.609438,3.044522,2.397895,1.386294,1.791759,1.098612,2,0.000000,4,6,0,0,36,2151,2,1,3,9.091895,24.0,0.099091,0.000900,6,1.098612,1.018131,0.333333,3.201632,0.004638,0,2018-01-04
4,4,1,10.973254,0,94,5.159055,3.784190,4.779123,3.850148,3.332205,2.772589,1.791759,5,1.386294,6,5,0,0,39,2173,4,1,3,9.874676,203.5,0.036192,0.003485,14,1.791759,2.351588,0.357143,4.472065,0.013711,0,2018-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,2,3,10.451638,1,8,5.298317,3.526361,4.110874,2.197225,1.386294,2.772589,1.791759,5,1.098612,5,8,0,1,49,2556,2,1,4,9.065430,151.5,0.048319,0.004369,12,1.791759,1.713615,0.416667,4.493228,0.007309,0,2022-11-03
1768,2,2,10.755880,1,20,5.036953,1.609438,4.043051,0.000000,2.302585,3.465736,1.609438,5,0.693147,4,8,0,0,53,2615,2,0,3,9.657310,111.0,0.130712,0.002364,10,1.609438,1.395793,0.400000,4.388563,0.007145,0,2022-11-04
1769,2,1,10.801512,1,6,5.932245,0.000000,3.663562,2.484907,2.197225,4.248495,1.791759,6,1.386294,6,6,0,0,47,2658,2,1,3,9.702941,216.5,0.147862,0.004400,15,1.791759,2.059819,0.333333,4.664441,0.011223,0,2022-11-05
1770,2,3,10.493383,0,54,3.850148,0.000000,3.433987,2.564949,0.693147,3.433987,0.693147,2,0.000000,4,6,0,0,37,2214,2,1,3,9.394826,44.5,0.290487,0.001233,6,0.693147,0.989160,0.166667,3.828373,0.004507,0,2022-11-06


In [9]:
pipe.transform(Xval)

,Education,Marital_Status,Income,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,idade,dias_cliente,education_n,parceiro,pessoas_casa,renda_per_capta,compra_total_ano,porcentagem_compras_gold,gasto_renda,total_purchases,disc_purchases,num_compras_ano,percent_disc_compra,ticket_medio,conversao_site,cupons_aceitados,date
1772,2,3,11.255436,1,46,6.054439,4.753590,5.680173,3.178054,3.583519,4.382027,0.693147,6,1.945910,11,3,0,0,41,2251,2,1,3,10.156850,445.0,0.085043,0.005740,23,0.693147,3.729454,0.043478,4.790158,0.026306,0,2022-11-08
1773,2,3,10.434910,1,11,2.944439,1.945910,2.772589,2.564949,2.197225,2.890372,1.386294,2,0.693147,3,5,0,0,70,2340,2,1,4,9.048703,29.5,0.253196,0.000867,6,1.386294,0.935897,0.500000,3.481872,0.005115,0,2022-11-09
1774,4,2,10.797921,1,21,3.970292,0.000000,2.302585,0.000000,0.000000,0.693147,1.098612,1,0.000000,4,4,0,0,57,2090,4,0,3,9.699350,30.5,0.016261,0.000623,5,1.098612,0.873206,0.400000,3.581538,0.003582,0,2022-11-10
1775,0,3,11.315973,0,89,6.599870,3.135494,5.860786,5.023881,3.258097,4.615121,0.693147,3,2.302585,13,1,0,0,58,2330,1,1,2,10.622838,641.0,0.075110,0.007775,25,0.693147,3.916309,0.040000,5.103971,0.037899,1,2022-11-11
1776,2,2,10.537362,0,82,3.555348,1.945910,3.091042,2.484907,1.609438,2.197225,0.693147,2,0.693147,3,6,0,0,44,2154,2,0,2,9.844242,38.0,0.100083,0.001008,6,0.693147,1.016713,0.166667,3.647415,0.004632,0,2022-11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211,2,1,11.022295,1,46,6.565265,3.784190,5.209486,3.761200,4.779123,5.513429,1.098612,9,1.386294,4,5,0,0,53,2393,2,1,3,9.923715,547.0,0.203575,0.008895,16,1.098612,2.440451,0.125000,5.416717,0.022315,0,2024-01-21
2212,4,3,11.066873,1,56,6.008813,0.000000,3.433987,0.000000,0.000000,2.197225,2.079442,8,1.098612,5,7,0,0,74,2031,4,1,5,9.457497,218.0,0.018182,0.003400,15,2.079442,2.695716,0.466667,4.405121,0.016740,1,2024-01-22
2213,2,0,10.950491,0,91,6.812345,3.891820,5.384495,3.496508,2.564949,3.218876,0.693147,2,1.386294,13,6,0,0,39,2167,2,0,1,10.950491,608.5,0.019529,0.010622,18,0.693147,3.031841,0.055556,5.306797,0.004604,1,2024-01-23
2214,3,3,11.145421,1,8,6.061457,3.433987,5.370638,4.394449,3.433987,4.127134,1.098612,6,1.791759,10,3,0,0,64,2168,3,1,3,10.046837,391.0,0.075112,0.005631,21,1.098612,3.535517,0.095238,4.714850,0.027299,0,2024-01-24


# Datetime features

In [13]:
feat = [

    'dia', 
    'mes', 

]

dict_events = {

     '2018-01-02':'pascoa',
     '2018-01-03':'bfriday',
     '2018-01-04':'natal',

     
    }


pipe = Pipeline([ 
    #('date_feat', DatetimeFeaturizer(feat)),
    ('dte', DaysToEvent('date',dict_events ))

])

pipe.fit_transform(Xtrain[['date']]).head()


----------- DaysToEvent



,date,dias_para_pascoa2018,dias_para_bfriday2018,dias_para_natal2018,dias_prox_evento,pascoa,bfriday,natal
0,2018-01-01,2,3,4,2.0,False,False,False
1,2018-01-02,1,2,3,1.0,True,False,False
2,2018-01-03,0,1,2,0.0,False,True,False
3,2018-01-04,-1,0,1,0.0,False,False,True
4,2018-01-05,-2,-1,0,0.0,False,False,False


In [16]:
df_fit = pd.DataFrame({
            'range_4': [1,2,3,4],
            'equal_0': [0,0,0,0],
            'equal_1': [1,1,1,1],
            'nan':[np.nan,np.nan,np.nan,1],
            'inf':[np.inf,np.inf,-np.inf,0]
        })


df_transform = pd.DataFrame({
            'range_4': [1,2,3,5],
            'equal_0': [0,0,0,1],
            'equal_1': [1,1,1,1],
            'nan':[np.nan,np.nan,np.nan,2],
            'inf':[np.inf,np.inf,-np.inf,1]
        })

pipe = Pipeline([ 
    ('sin_cos', SinCosFeaturizer())
])

r_fit = pipe.fit_transform(df_fit)
r_transform = pipe.transform(df_transform)


----------- SinCosFeaturizer



In [17]:
r_fit

,equal_0_cos,equal_0_sen,equal_1_cos,equal_1_sen,inf_cos,inf_sen,nan_cos,nan_sen,range_4_cos,range_4_sen
0,NaN,NaN,1.0,-2.449294e-16,NaN,NaN,NaN,NaN,6.123234e-17,1.000000e+00
1,NaN,NaN,1.0,-2.449294e-16,NaN,NaN,NaN,NaN,-1.000000e+00,1.224647e-16
2,NaN,NaN,1.0,-2.449294e-16,NaN,NaN,NaN,NaN,-1.836970e-16,-1.000000e+00
3,NaN,NaN,1.0,-2.449294e-16,1.0,0.0,1.0,-2.449294e-16,1.000000e+00,-2.449294e-16


# rolling window

In [19]:
rw = pd.DataFrame({
    
    'dt': pd.date_range(start='1/1/2018', periods=8).tolist() + pd.date_range(start='1/1/2018', periods=8).tolist(),
    'pdv':['a','a','a','a','a','a','a','a','b','b','b','b','b','b','b','b'],
    'sku':['XXX','XXX','XXX','XXX','ZZZ','ZZZ','ZZZ','ZZZ','XXX','XXX','XXX','XXX','ZZZ','ZZZ','ZZZ','ZZZ'],
    'vol':[1]*16
})
rw

,dt,pdv,sku,vol
0,2018-01-01,a,XXX,1
1,2018-01-02,a,XXX,1
2,2018-01-03,a,XXX,1
3,2018-01-04,a,XXX,1
4,2018-01-05,a,ZZZ,1
5,2018-01-06,a,ZZZ,1
6,2018-01-07,a,ZZZ,1
7,2018-01-08,a,ZZZ,1
8,2018-01-01,b,XXX,1
9,2018-01-02,b,XXX,1


In [20]:
date_column = 'dt'
target = 'vol'
time_ranges = [2,3]
rol_functions = [

                'mean',
                'median',
                'sum',
                'std',
                'max',
                'min',
                'kurtosis',
                'skewness',
                'quant_90',
                'quant_10',
                'trend',
                'longest_period_above_mean',
                'longest_period_below_mean',
                'longest_consecutive_decreases',
                'longest_consecutive_increases',
                'last_max',
                'last_min',
                'size',
                'list'

        ]
grouped_variables = [ ['sku']]#, ['pdv'], ['pdv', 'sku']]


pipe = Pipeline([ 
    ('rolling', WindowFeatures(date_column, target, time_ranges, rol_functions, grouped_variables, print_ = False))

])

pipe.fit_transform(rw).head()

erro ao computar list 2
must be real number, not list
erro ao computar list 3
must be real number, not list
window 0.21117877960205078


,dt,pdv,sku,vol,vol_2_sku_mean,vol_2_sku_median,vol_2_sku_sum,vol_2_sku_std,vol_2_sku_max,vol_2_sku_min,vol_2_sku_kurtosis,vol_2_sku_skewness,vol_2_sku_quant_90,vol_2_sku_quant_10,vol_2_sku_trend,vol_2_sku_longest_period_above_mean,vol_2_sku_longest_period_below_mean,vol_2_sku_longest_consecutive_decreases,vol_2_sku_longest_consecutive_increases,vol_2_sku_last_max,vol_2_sku_last_min,vol_2_sku_size,vol_3_sku_mean,vol_3_sku_median,vol_3_sku_sum,vol_3_sku_std,vol_3_sku_max,vol_3_sku_min,vol_3_sku_kurtosis,vol_3_sku_skewness,vol_3_sku_quant_90,vol_3_sku_quant_10,vol_3_sku_trend,vol_3_sku_longest_period_above_mean,vol_3_sku_longest_period_below_mean,vol_3_sku_longest_consecutive_decreases,vol_3_sku_longest_consecutive_increases,vol_3_sku_last_max,vol_3_sku_last_min,vol_3_sku_size
0,2018-01-01,a,XXX,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,b,XXX,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,a,XXX,1,2.0,2.0,4.0,0.0,2.0,2.0,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-02,b,XXX,1,2.0,2.0,4.0,0.0,2.0,2.0,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-03,a,XXX,1,2.0,2.0,4.0,0.0,2.0,2.0,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,6.0,0.0,2.0,2.0,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0


# Tf-idf / CountVectorizer

In [23]:
tf = pd.DataFrame({'texto':['oi oi oi como vai voce', 'eu vou bem', 'e voce']})

args = { 'max_features': 20}
pipe = Pipeline([ 
    ('tfidf', TfIdfPandas(**args))
    #('cv', CountVectorizerPandas(**args))

])

pipe.fit_transform(tf).head()


----------- TfIdfPandas



,bem,como,eu,oi,vai,voce,vou
0,0.00000,0.378436,0.00000,0.79419,0.378436,0.28781,0.00000
1,0.57735,0.000000,0.57735,0.00000,0.000000,0.00000,0.57735
2,0.00000,0.000000,0.00000,0.00000,0.000000,1.00000,0.00000


In [14]:
tf.select_dtypes(include=['category', 'object'])

,texto
0,oi oi oi como vai voce
1,eu vou bem
2,e voce


In [15]:
pipe = Pipeline([ 
    ('le', LabelEncoderPandas('texto'))

])

pipe.fit_transform(tf).head()

,texto
0,2
1,1
2,0


# Target encoder

In [18]:
df_ = df[df['Marital_Status'].isin(['Single', 'Together'])].copy()
df_['Marital_Status'] = df_['Marital_Status'].astype('str').astype('category')

In [19]:
pipe = Pipeline([ 
    ('te', TargetEncoderPandas('Marital_Status', df_['Response'], operation = 'mean', fill_unknow = 'none' ))

])

In [20]:
pipe.fit_transform(df_).head()
# {'Single': 0.2280334728033473, 'Together': 0.10471204188481675}

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,idade,dias_cliente,education_n,parceiro,pessoas_casa,renda_per_capta,compra_total_ano,porcentagem_compras_gold,gasto_renda,total_purchases,disc_purchases,num_compras_ano,percent_disc_compra,ticket_medio,conversao_site,cupons_aceitados,date
0,Graduation,0.228033,10.970592,0,0,58,6.455199,4.488636,6.304449,5.153292,4.488636,4.488636,1.386294,8,2.397895,4,7,0,1,63,2675,2,0,1,10.970592,764.5,0.055959,0.013064,22,1.386294,3.001869,0.136364,5.543906,0.012736,0,2018-01-01
1,Graduation,0.228033,10.743869,1,1,38,2.484907,0.693147,1.945910,1.098612,0.693147,1.945910,1.098612,1,0.693147,2,5,0,0,66,2125,2,0,3,9.645300,10.5,0.251314,0.000227,4,1.098612,0.687059,0.500000,2.790093,0.002820,0,2018-01-02
2,Graduation,0.104712,11.179046,0,0,26,6.056784,3.912023,4.852030,4.718499,3.091042,3.761200,0.693147,8,1.098612,10,4,0,0,55,2324,2,1,2,10.485913,367.0,0.055643,0.005112,20,0.693147,3.141136,0.050000,4.769300,0.025490,0,2018-01-03
3,Graduation,0.104712,10.190432,1,0,26,2.484907,1.609438,3.044522,2.397895,1.386294,1.791759,1.098612,2,0.000000,4,6,0,0,36,2151,2,1,3,9.091895,24.0,0.099091,0.000900,6,1.098612,1.018131,0.333333,3.201632,0.004638,0,2018-01-04
5,Master,0.104712,11.043146,0,1,16,6.255750,3.761200,4.595120,0.000000,3.761200,2.708050,1.098612,6,1.609438,10,6,0,0,53,2305,3,1,3,9.944566,351.0,0.019747,0.005599,20,1.098612,3.167028,0.100000,4.716975,0.012931,0,2018-01-06


In [21]:
df_.head()

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,idade,dias_cliente,education_n,parceiro,pessoas_casa,renda_per_capta,compra_total_ano,porcentagem_compras_gold,gasto_renda,total_purchases,disc_purchases,num_compras_ano,percent_disc_compra,ticket_medio,conversao_site,cupons_aceitados,date
0,Graduation,Single,10.970592,0,0,58,6.455199,4.488636,6.304449,5.153292,4.488636,4.488636,1.386294,8,2.397895,4,7,0,1,63,2675,2,0,1,10.970592,764.5,0.055959,0.013064,22,1.386294,3.001869,0.136364,5.543906,0.012736,0,2018-01-01
1,Graduation,Single,10.743869,1,1,38,2.484907,0.693147,1.945910,1.098612,0.693147,1.945910,1.098612,1,0.693147,2,5,0,0,66,2125,2,0,3,9.645300,10.5,0.251314,0.000227,4,1.098612,0.687059,0.500000,2.790093,0.002820,0,2018-01-02
2,Graduation,Together,11.179046,0,0,26,6.056784,3.912023,4.852030,4.718499,3.091042,3.761200,0.693147,8,1.098612,10,4,0,0,55,2324,2,1,2,10.485913,367.0,0.055643,0.005112,20,0.693147,3.141136,0.050000,4.769300,0.025490,0,2018-01-03
3,Graduation,Together,10.190432,1,0,26,2.484907,1.609438,3.044522,2.397895,1.386294,1.791759,1.098612,2,0.000000,4,6,0,0,36,2151,2,1,3,9.091895,24.0,0.099091,0.000900,6,1.098612,1.018131,0.333333,3.201632,0.004638,0,2018-01-04
5,Master,Together,11.043146,0,1,16,6.255750,3.761200,4.595120,0.000000,3.761200,2.708050,1.098612,6,1.609438,10,6,0,0,53,2305,3,1,3,9.944566,351.0,0.019747,0.005599,20,1.098612,3.167028,0.100000,4.716975,0.012931,0,2018-01-06


In [22]:
pipe.transform(df)['Marital_Status']

0       0.228033
1       0.228033
2       0.104712
3       0.104712
4            NaN
          ...   
2211         NaN
2212    0.104712
2213         NaN
2214    0.104712
2215         NaN
Name: Marital_Status, Length: 2216, dtype: float64

In [23]:
df['Marital_Status']

0         Single
1         Single
2       Together
3       Together
4        Married
          ...   
2211     Married
2212    Together
2213    Divorced
2214    Together
2215     Married
Name: Marital_Status, Length: 2216, dtype: category
Categories (5, object): ['Divorced', 'Married', 'Single', 'Together', 'Widow']